In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
import sys
sys.path.append('../')
from gis_resources import san_diego_county_zips
import os
from arcgis.features import GeoAccessor

In [2]:
pd.set_option('display.max_colwidth', None)


In [3]:

#gis = GIS(username='izaslavsky_ucsd')  # this will ask for password. 
print("Enter the Client ID/APP ID generated from Arc GIS:")
app_id = input()
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)
arcgis.__version__
# Elm5V3upnnV17Q3r

Enter the Client ID/APP ID generated from Arc GIS:


 Elm5V3upnnV17Q3r


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=MB6SnIghRE9j3BTbadbHhXCRXLzfZA&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'2.1.0'

In [4]:
# df = pd.DataFrame(
#     {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
#      'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
#      'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
#      'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})

In [5]:
# gdf = geopandas.GeoDataFrame(
#     df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [6]:
# print(gdf.head())

In [7]:
# world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# # We restrict to South America.
# ax = world[world.continent == 'South America'].plot(
#     color='white', edgecolor='black')

# # We can now plot our ``GeoDataFrame``.
# gdf.plot(ax=ax, color='red')

# plt.show()

In [8]:
# df

In [ ]:
print("Nourish User:")
nourish_user = input()
print("Nourish Password:")
nourish_pswd = input()

In [10]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [11]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from ca_business')

        # display the PostgreSQL database server version
ca_business_result = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [12]:
ca_business_result_list = [(row[1],row[4],row[5],row[7],row[8],row[9]) for row in ca_business_result]
#lat_long_list[:10]

In [33]:
# Create a DataFrame
dataFrame = pd.DataFrame(ca_business_result_list,
              columns=("business_name","latitude", "longitude","zip", "categories","city")
              );

In [35]:
# since some of the zip are not present in ca_business table
dataFrame = dataFrame[~dataFrame['zip'].isna()]

In [37]:

dataFrame = dataFrame.astype({"zip": int})
dataFrame = dataFrame.astype({"zip": str})

In [38]:
# Pick the data for only San Diego County
san_diego_county_zips_lst = san_diego_county_zips()
#df[df['col1'].isin(['a', 'c', 'h'])]
san_diego_county_zips_lst[:10]

['91910',
 '91911',
 '91913',
 '91914',
 '91915',
 '91916',
 '91917',
 '91931',
 '91932',
 '91934']

In [39]:
dataFrame = dataFrame[dataFrame["zip"].isin(san_diego_county_zips_lst)]
dataFrame

,business_name,latitude,longitude,zip,categories,city
9,Bird Park (playground),32.7406017,-117.1347167,92104,[Playground],San Diego
21,San Diego Regional Center,32.6586542,-117.1033054,91950,[Non-profit organization],National City
33,Phở Duyên Mai,32.8305101,-117.1388964,92123,[Vietnamese restaurant],San Diego
35,Lobster West,33.0420054,-117.2930713,92024,"[Restaurant, Sandwich shop, Seafood restaurant]",Encinitas
45,Panda Express,33.128009899999995,-117.09391099999999,92025,"[Chinese restaurant, Asian restaurant, Restaurant]",Escondido
...,...,...,...,...,...,...
515891,Haircut Place,32.7032124,-117.1000529,92113,"[Beauty salon, Hair salon]",San Diego
515907,Evans Tire & Service Centers,32.9561091,-117.044693,92064,"[Tire shop, Auto repair shop, Brake shop, Oil change service, Wheel alignment service]",Poway
515941,Lake Calavera Hiking Trail,33.1692003,-117.2851578,92010,"[Hiking area, Tourist attraction]",Carlsbad
515943,Banfield Pet Hospital,33.1722239,-117.2144568,92083,"[Veterinarian, Animal hospital]",Vista


In [40]:
#ca_business_sgdf.head(5)
dataFrame['categories'].fillna('',inplace=True)
dataFrame['categories']=dataFrame.categories.apply(lambda x: ','.join(x))

In [41]:
#!pip install psycopg2-binary

In [42]:
#!pip show psycopg2-binary

In [43]:
gdf = geopandas.GeoDataFrame(
    dataFrame, geometry=geopandas.points_from_xy(dataFrame.longitude, dataFrame.latitude))

In [44]:
ca_business_sgdf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

In [45]:
#ca_business_sgdf=ca_business_sgdf.drop(['latitude','longitude'], axis=1)

In [46]:
#ca_business_sgdf['zip'] = ca_business_sgdf['zip'].astype(str)

In [47]:
#ca_business_sgdf.info()

In [48]:
%%time
# Convert back from a SEDF into a feature layer, and publishing on AGOL
my_new_featurelayer = ca_business_sgdf.spatial.to_featurelayer(title="San Diego County Businesses", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['SanDiegoCountyBusinesses'])

CPU times: user 35.3 s, sys: 2.21 s, total: 37.5 s
Wall time: 1min 21s


In [28]:
#ca_business_sgdf